In [1]:
author="Jumabek Alikhanov"
date = 'Nov 19,2019'

In [2]:
#imports
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
%load_ext autoreload

In [3]:
# load data
dataroot = 'MachineLearningCVE/'
SEED=5

In [4]:
%autoreload 2
from preprocessing import load_data
X,y = load_data(dataroot) # reads csv file and returns np array of X,y -> of shape (N,D) and (N,1)

MachineLearningCVE/*.pcap_ISCX.csv


/home/juma/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/juma/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


2830743 flow records read which has 79 feature dimension


In [5]:
from sklearn import preprocessing
from preprocessing import balance_data, encode_label, normalize

X = normalize(X)


In [6]:
# lets try simple RF classifier performance from sklearn

from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED)


In [9]:
# Random Foresty with 10 DecisionTrees
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
K=5
X = X_train
Y = y_train
results = {}
skf = StratifiedKFold(n_splits=K,random_state=SEED)

for fold_index, (dev_index,test_index) in enumerate(skf.split(X,Y)): 
    print('---------------------------------------------')
    print('Fold #{}'.format(fold_index))    
    clf = RandomForestClassifier(n_estimators=10, random_state=SEED)
    X_dev = X[dev_index]
    y_dev = Y[dev_index]
    X_test = X[test_index]
    y_test = Y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    for train_index, val_index in sss.split(X_dev, y_dev):
        print(train_index,)
    
    X_train = X[train_index]
    y_dev = Y[train_index]
    X_val = X[val_index]
    val = Y[val_index]

    
    X_train,y_train = balance_data(X_train,y_train)

    clf.fit(X_train,y_train)
    
    pred = clf.predict(X_val)
    acc = metrics.accuracy_score(y_val,pred)*100        
    balanced_acc = metrics.balanced_accuracy_score(y_val,pred)*100

    pred = clf.predict(X_test)
    test_acc = metrics.balanced_accuracy_score(y_test,pred)*100
    print('val acc:',acc)
    print('balanced val acc: ',balanced_acc)
    print('balanced test acc: ',test_acc)
    
    results[fold_index]= (acc,balanced_acc,test_acc)
    


---------------------------------------------
Fold #0
<generator object BaseShuffleSplit.split at 0x7f9b80092048>


NameError: name 'train_index' is not defined

In [ ]:
sum_acc = 0
sum_balanced_acc = 0
sum_test_acc = 0
for fold_index,res in results.items():
    (acc,balanced_acc,test_acc) = res
    sum_acc+=acc
    sum_balanced_acc +=balanced_acc
    sum_test_acc+= test_acc
print("{}-fold evaluation:".format(K))
print('val acc:',sum_acc/K)
print('balanced val acc: ',sum_balanced_acc/K)
print('balanced test acc: ',sum_test_acc/K)
print('{0:.2f} {1:.2f} {2:.2f}'.format(sum_acc/K,sum_balanced_acc/K,sum_test_acc/K))